Import Libraries:

In [1]:
from CSVExport import *
from FineTuningPipeline import * 
from GPTClassifier import * 
from PerformanceAnalyzer import * 

In [2]:
import os
from pymongo import MongoClient
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
from collections import Counter
import requests
import re
import openai
import json
import tiktoken 
import numpy as np
from collections import defaultdict
import chime
import time

In [3]:
disorder_list = ["ADHD", "PTSD", "OCD", "ANXIETY", "DEPRESSION", "CONTROL", "AUTISM", "BIPOLAR","EATING DISORDER", "SCHIZOPHRENIA"]
df = pd.DataFrame(columns=['_id', 'sampled_tweets', 'ground_truth'])

for disorder in disorder_list:
    temp = create_df_from_db_limited(disorder, 10) # The 10 indicates the number of samples from the database, set it to a very high value to obtain all samples per class
    df = pd.concat([df, temp], ignore_index=True)
    

/Users/lukasschmidt/Desktop/Twitter/CSVExport.py:106: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '"@USER Save/invest some of it for gifts for close loved ones then spend the rest on creating memorable experiences with and for others.";"I am a greatly likable person (86.43%). HTTPURL via @USER";"It's easy to judge the mistakes of others, but it's difficult to realize one's own mistakes.";"@USER Happy Birthday dude! Hope it's a good one. 🎉 HTTPURL";"You always have a choice. ️ HTTPURL";"@USER That IS a clever title!";"@USER Amen to that! The phrase, "what goes around comes around" still holds true.";"I'm thankful for all the difficult people who have come and gone in my life. They were all lessons learned in who I do not want to be.";"Comparison is a thief of joy.";"@USER Yeah the hardest part is having the discipline to do it. I've discovered that keeping it short (10 minutes) and doing it first thing helps. That 

In [5]:
df.ground_truth.unique()

array(['ADHD', 'PTSD', 'OCD', 'ANXIETY', 'DEPRESSION', 'CONTROL',
       'AUTISM', 'BIPOLAR', 'EATING DISORDER', 'SCHIZOPHRENIA'],
      dtype=object)

Before doing any further steps we want to clean the dataset in order to not have any easy to spot signal words inside for the classifier:

In [7]:
df = clean_data(df)

Create a safety copy of the dataframe:

In [6]:
safe = df

In [32]:
df = safe

We need to sample the training data from our whole dataset:
CAUTION: This will drop samples from the dataframe returned from querying !

In [33]:
import pandas as pd

# Assuming you have the dataframe df with the sampled data

disorder_list = ["ADHD", "PTSD", "OCD", "ANXIETY", "DEPRESSION", "CONTROL", "AUTISM", "BIPOLAR", "EATING DISORDER", "SCHIZOPHRENIA"]
data = pd.DataFrame(columns=['_id', 'sampled_tweets', 'ground_truth'])

for disorder in disorder_list:
    disorder_data = df[df['ground_truth'] == disorder]
    sample_size = int(len(disorder_data) * 0.1)
    sampled_data = disorder_data.sample(n=sample_size)
    sampled_data['ground_truth'] = disorder
    data = pd.concat([data, sampled_data], ignore_index=True)

# Merge the original dataframe with the sampled dataframe on the index
df = df.merge(data, how='left', indicator=True).query('_merge == "left_only"').drop('_merge', axis=1)

# Display the updated data and df dataframes
print(data)
print(df)


            _id                                     sampled_tweets  \
0  7.993960e+05  "@USER Save/invest some of it for gifts for cl...   
1  1.733508e+09  "Im sorry, why is no one talking about "nope" ...   
2  1.249942e+08  "@USER I followed you pre bitty but that lil c...   
3  4.091108e+09  "Yoongi, please, don't look IN a very long tim...   
4  1.748346e+07  "#Olympic #IceHockey silver medallist Ivan Fed...   
5  2.035621e+06  "All the Sexual Banter, None of the Sexual Ten...   
6  1.422963e+07  "At The #Hospital by @USER HTTPURL #Life #Life...   
7  1.465121e+18  "Well, it was fun while it lasted. HTTPURL";"H...   
8  1.459690e+18  "pro tip: don't get a vape flavor of your fav ...   
9  7.280914e+17  "I belive telling lies to police is a crime on...   

      ground_truth  
0             ADHD  
1             PTSD  
2              OCD  
3          ANXIETY  
4       DEPRESSION  
5          CONTROL  
6           AUTISM  
7          BIPOLAR  
8  EATING DISORDER  
9    SCHIZOPHRENIA 

/var/folders/yq/4w2x66q12594sgmnxblsz_bh0000gn/T/ipykernel_11402/83071887.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, sampled_data], ignore_index=True)


In [26]:
df.to_csv("DemoTest.csv")
data.to_csv("DemoTrain.csv")

In [32]:
df = pd.read_csv("DemoTest.csv")
data = pd.read_csv("DemoTrain.csv")


We will now apply a mapping to make the data suitable for binary classification:

In [89]:
mapping = {'PTSD': 'psychological disorder', 
           'SCHIZOPHRENIA': 'psychological disorder',
           'EATING DISORDER': 'psychological disorder',
           'BIPOLAR': 'psychological disorder', 
           'AUTISM': 'psychological disorder',
           'ADHD': 'psychological disorder',
           "OCD" : "psychological disorder",
           "DEPRESSION": "psychological disorder", 
           "ANXIETY" :  "psychological disorder",
           "CONTROL" :  "No psychological disorder"}

df["binary_disorder"] = df["ground_truth"].map(mapping)
data["binary_disorder"] = data["ground_truth"].map(mapping)


In [38]:
df["GPT35Finetuned"] = ""
df["GPT35Finetuned_Binary"] = ""
df["GPT35TurboBaseTopP1_Binary"] = ""
df["GPT35TurboBaseTemp0_Binary"] = ""
df["Prediction_GPT4_Temp0"] = ""
df["GPT35TurboBaseTemp0_Multi"] = ""
df["GPT35Finetuned_temp"] = ""


In [39]:
specific_values_multi = ["ADHD", "PTSD", "OCD", "AUTISM", "BIPOLAR","EATING DISORDER", "SCHIZOPHRENIA"] 
multi_df = df[df['ground_truth'].isin(specific_values_multi)]
train_multi = data[data['ground_truth'].isin(specific_values_multi)]

This will create an input file for the fine tuning, the ending has to be jsonl:

In [4]:
output_file_binary = "binary_output_train.jsonl"
output_file_multi = "multi_output_train.jsonl"
fine_tuning_pipeline_binary(data, output_file_binary)
fine_tuning_pipeline_multi(train_multi, output_file_multi)

In [5]:
data_path = output_file_binary
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]
    print("Worked")
    chime.theme('material')
    chime.success()

Worked


This will check the dataset for potential errors and give a rough estimation of training: 

In [6]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 390, 888
mean / median: 536.8, 496.5
p5 / p95: 401.7, 691.8

#### Distribution of num_assistant_tokens_per_example:
min / max: 1, 7
mean / median: 2.9, 2.0
p5 / p95: 1.9, 4.299999999999999

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [7]:
data_path = output_file_multi
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]
    print("Worked")
    chime.theme('material')
    chime.success()

Worked


In [8]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 464, 962
mean / median: 610.8, 570.5
p5 / p95: 475.7, 765.8

#### Distribution of num_assistant_tokens_per_example:
min / max: 1, 7
mean / median: 2.9, 2.0
p5 / p95: 1.9, 4.299999999999999

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


This will start the actual fine tuning job, it only needs the name of the input file to work:

finetune_job

In [8]:
finetune_job(output_file_binary)

Start fine tuning
Train file created
Wait for fine tuning to be finished
FineTuningJob(id='ftjob-yhncZxbI7b2Sdiz0nRPBp0kd', created_at=1705625685, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-sBvQNDCDEu72tVa8v9bZtyfM', result_files=[], status='validating_files', trained_tokens=None, training_file='file-5ZUs8GdCOjonE7pYr0hF2Aac', validation_file=None)


There is sadly no convenient way to automate the check of the fine tuning process, you will get an email notification, when the account is linked to your OpenAI account regarding the process of fine-tuning. The link for checking can be found here: https://platform.openai.com/finetune. You may manually check the process with the cell below : 

In [14]:
load_dotenv()
api_key  = os.getenv("API")
client = OpenAI(api_key = api_key)
client.fine_tuning.jobs.retrieve("ftjob-SQTBi5bvEp6uLTjwxpbS57pc") # You have to paste your FineTuningJob model id here, it will be printed by the cell above. FineTuningJob(id='ftjob-yhncZxbI7b2Sdiz0nRPBp0kd

Please give the finetune pipeline enough time to process, it may take 20-30 minutes depending on the servers:


The process of fine-tuning can be seen here : https://platform.openai.com/finetune , in case the python file does not work the fine tuning can also be started using the GUI on the website

In [ ]:
time.sleep(1200)

In [ ]:
finetune_job(output_file_multi)

In [ ]:
load_dotenv()
api_key  = os.getenv("API")
client = OpenAI(api_key = api_key)
client.fine_tuning.jobs.retrieve("") # You have to paste your FineTuningJob model id here, it will be printed by the cell above. FineTuningJob(id='ftjob-yhncZxbI7b2Sdiz0nRPBp0kd

Please paste your fine_tuned model id here, it can be retreived from the cell above: INFORMATION: If you want to use my fine-tuned model, please just ask me and I'll share my account information happily :). Unfortunately there is currently no feature made by OpenAI to share fine-tuned models.

In [4]:
fine_tuned_model = "ft:gpt-3.5-turbo-1106:personal::8hmyAfAD"
fine_tuned_binary = "ft:gpt-3.5-turbo-0613:personal::8gZJqlav"

In [6]:
df

,Unnamed: 0,_id,sampled_tweets,ground_truth,binary_disorder
0,10,9.777339e+17,"""@USER @USER me every allergy season"";""@USER O...",PTSD,psychological disorder
1,11,1.361157e+18,"""between monkey pox and covid i think i need t...",PTSD,psychological disorder
2,12,7.146602e+17,"""every track on this new tape produced by your...",PTSD,psychological disorder
3,13,1.403116e+18,"""@USER @USER @USER I'm well ty! 🫂"";""@USER For ...",PTSD,psychological disorder
4,14,4.121971e+08,"""@USER @USER @USER @USER @USER 🥱 🥱 The most b...",PTSD,psychological disorder
...,...,...,...,...,...
85,95,1.026962e+18,"""Im at Paradise Lynwood FRIDAY 8pm-Midnight gi...",SCHIZOPHRENIA,psychological disorder
86,96,1.334662e+18,"""@USER we say whatever the FUCK we like around...",SCHIZOPHRENIA,psychological disorder
87,97,8.010677e+08,"""jesse x bullseye otp"";""now, i'm no homophobe""...",SCHIZOPHRENIA,psychological disorder
88,98,1.253349e+18,"""mes oc's HTTPURL"";""them HTTPURL"";""@USER mon r...",SCHIZOPHRENIA,psychological disorder


In [45]:
%%capture
gpt_classifier_ft(multi_df, fine_tuned_model);

In [46]:
%%capture
gpt_classifier_ft_binary(df, fine_tuned_binary)

In [47]:
%%capture
gpt_classifier_turbo_binary(df)

In [48]:
%%capture
gpt_classifier_turbo_binary_temp(df)

In [49]:
%%capture
gpt_classifier_4(multi_df)

In [57]:
multi_df["GPT35TurboBaseTopPMulti"] = ""


/var/folders/yq/4w2x66q12594sgmnxblsz_bh0000gn/T/ipykernel_11402/3713738311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_df["GPT35TurboBaseTopPMulti"] = ""


In [7]:

%%capture

gpt_classifier_ft_temp(multi_df, fine_tuned_model)

In [59]:
%%capture
gpt_classifier_turbo_multi_top(multi_df)

In [62]:
%%capture
gpt_classifier_turbo_multi_temp0(multi_df)

In [67]:
multi_df.columns

Index(['_id', 'sampled_tweets', 'ground_truth', 'binary_disorder',
       'GPT35Finetuned', 'GPT35Finetuned_Binary', 'GPT35TurboBaseTopP1_Binary',
       'GPT35TurboBaseTemp0_Binary', 'Prediction_GPT4_Temp0',
       'GPT35TurboBaseTemp0_Multi', 'GPT35TurboBaseTopPMulti'],
      dtype='object')

In [68]:
df.columns

Index(['_id', 'sampled_tweets', 'ground_truth', 'binary_disorder',
       'GPT35Finetuned', 'GPT35Finetuned_Binary', 'GPT35TurboBaseTopP1_Binary',
       'GPT35TurboBaseTemp0_Binary', 'Prediction_GPT4_Temp0',
       'GPT35TurboBaseTemp0_Multi'],
      dtype='object')

In [77]:
multi_df["GPT35Finetuned"] = multi_df["GPT35Finetuned"].apply(lambda x: x.split(':')[-1].strip() if ':' in x else x)
multi_df["Prediction_GPT4_Temp0"] = multi_df["Prediction_GPT4_Temp0"].apply(lambda x: x.split(':')[-1].strip() if ':' in x else x)
multi_df["GPT35TurboBaseTemp0_Multi"] = multi_df["GPT35TurboBaseTemp0_Multi"].apply(lambda x: x.split(':')[-1].strip() if ':' in x else x)
multi_df["GPT35TurboBaseTopPMulti"] = multi_df["GPT35TurboBaseTopPMulti"].apply(lambda x: x.split(':')[-1].strip() if ':' in x else x)


In [79]:
columns_to_drop = ['GPT35Finetuned_Binary', 'GPT35TurboBaseTopP1_Binary', 'GPT35TurboBaseTemp0_Binary']
multi_df = multi_df.drop(columns=columns_to_drop)

In [81]:
multi_df.ground_truth.unique()

array(['ADHD', 'PTSD', 'OCD', 'AUTISM', 'BIPOLAR', 'EATING DISORDER',
       'SCHIZOPHRENIA'], dtype=object)

In [11]:
columns_to_process = ['Prediction_GPT4_Temp0', 'GPT35TurboBaseTemp0_Multi', "GPT35TurboBaseTopPMulti", "GPT35Finetuned", "GPT35Finetuned_temp"]

# List of substrings to check for
disorder_list = ['ADHD', 'PTSD', 'OCD', 'AUTISM', 'BIPOLAR', 'EATING DISORDER', 'SCHIZOPHRENIA']

def extract_disorder(text):
    for disorder in disorder_list:
        if disorder in text:
            return disorder
    return text

# Apply the function only to the selected columns
for column in columns_to_process:
    multi_df[column] = multi_df[column].apply(extract_disorder)


In [85]:
columns_to_drop = ["Prediction_GPT4_Temp0", "GPT35TurboBaseTemp0_Multi", "GPT35Finetuned"]
df = df.drop(columns=columns_to_drop)

In [87]:
df["GPT35Finetuned_Binary"] = df["GPT35Finetuned_Binary"].apply(lambda x: x.lower() if isinstance(x, str) else x)
df["GPT35TurboBaseTopP1_Binary"] = df["GPT35Finetuned_Binary"].apply(lambda x: x.lower() if isinstance(x, str) else x)
df["GPT35TurboBaseTemp0_Binary"] = df["GPT35TurboBaseTemp0_Binary"].apply(lambda x: x.lower() if isinstance(x, str) else x)




In [91]:
df["binary_disorder"] = df["binary_disorder"].apply(lambda x: x.lower() if isinstance(x, str) else x)


In [94]:
multi_df

,Unnamed: 0,_id,sampled_tweets,ground_truth,binary_disorder,GPT35Finetuned,Prediction_GPT4_Temp0,GPT35TurboBaseTemp0_Multi,GPT35TurboBaseTopPMulti
0,1,2.716031e+06,"""great i have... another? the same? ear infect...",ADHD,psychological disorder,ADHD,"I'm sorry, but I cannot provide a diagnosis or...",PTSD,BIPOLAR
1,2,8.481222e+06,"""People should always come before profits. Bec...",ADHD,psychological disorder,PTSD,"I'm sorry, but it is not appropriate or ethica...",PTSD,PTSD
2,3,1.010535e+07,"""Black lives still matter. COVID isnt over. ...",ADHD,psychological disorder,ADHD,"I'm sorry, but I cannot provide a diagnosis or...",PTSD,PTSD
3,4,1.422955e+07,"""@USER I love the ride but that pre show and t...",ADHD,psychological disorder,ADHD,EATING DISORDER,BIPOLAR,AUTISM
4,5,1.439176e+07,"""@USER I am 100% certain this is exactly what ...",ADHD,psychological disorder,ADHD,"I'm sorry, but I cannot provide a diagnosis or...",PTSD,PTSD
...,...,...,...,...,...,...,...,...,...
58,94,2.314623e+09,"""@USER Its about the schiz. You know the 7/22 ...",SCHIZOPHRENIA,psychological disorder,SCHIZOPHRENIA,SCHIZOPHRENIA,SCHIZOPHRENIA,SCHIZOPHRENIA
59,96,1.404918e+18,"""I like how my brother's friends always have h...",SCHIZOPHRENIA,psychological disorder,PTSD,EATING DISORDER,PTSD,PTSD
60,97,2.377368e+09,"""""Elevated odds of multiple conditions at the ...",SCHIZOPHRENIA,psychological disorder,ADHD,SCHIZOPHRENIA,PTSD,PTSD
61,98,8.010677e+08,"""jesse x bullseye otp"";""now, i'm no homophobe""...",SCHIZOPHRENIA,psychological disorder,ADHD,"I'm sorry, but I cannot assist with this request.",SCHIZOPHRENIA,OCD


In [96]:
multi_df.to_csv("MultiDemo0358.csv")
df.to_csv("BinaryDemo0358.csv")


In [3]:
multi_df = pd.read_csv("MultiDemo0358.csv")
df = pd.read_csv("BinaryDemo0358.csv")

Performance of GPT35Finetuned_Binary :

Sometimes NaN values will show for the performance metrics on smaller selected datasets compared to the one that I chose for the thesis. Simply due to not having enough data.

In [5]:
calculate_performance_binary(df, "GPT35Finetuned_Binary")

no psychological disorder
Precision  0.16
Recall: 0.8888888888888888
F1 0.27118644067796616
TPR 0.8888888888888888
FPR 0.5185185185185185
AUC 0.631578947368421
psychological disorder
Precision  0.975
Recall: 0.48148148148148145
F1 0.6446280991735537
TPR 0.48148148148148145
FPR 0.1111111111111111
AUC 0.8125


,Disorder,Precision,Recall,F1,AUC
0,no psychological disorder,0.160,0.888889,0.271186,0.631579
1,psychological disorder,0.975,0.481481,0.644628,0.812500


Performance of GPT35TurboBaseTopP1_Binary:

In [6]:
calculate_performance_binary(df, "GPT35TurboBaseTopP1_Binary")

no psychological disorder
Precision  0.16
Recall: 0.8888888888888888
F1 0.27118644067796616
TPR 0.8888888888888888
FPR 0.5185185185185185
AUC 0.631578947368421
psychological disorder
Precision  0.975
Recall: 0.48148148148148145
F1 0.6446280991735537
TPR 0.48148148148148145
FPR 0.1111111111111111
AUC 0.8125


,Disorder,Precision,Recall,F1,AUC
0,no psychological disorder,0.160,0.888889,0.271186,0.631579
1,psychological disorder,0.975,0.481481,0.644628,0.812500


In [7]:
calculate_performance_binary(df, "GPT35TurboBaseTemp0_Binary")

no psychological disorder
Precision  0.3333333333333333
Recall: 0.2222222222222222
F1 0.26666666666666666
TPR 0.2222222222222222
FPR 0.04938271604938271
AUC 0.8181818181818182
psychological disorder
Precision  0.9166666666666666
Recall: 0.9506172839506173
F1 0.9333333333333332
TPR 0.9506172839506173
FPR 0.7777777777777778
AUC 0.5499999999999999


,Disorder,Precision,Recall,F1,AUC
0,no psychological disorder,0.333333,0.222222,0.266667,0.818182
1,psychological disorder,0.916667,0.950617,0.933333,0.550000


In [8]:
calculate_performance_multi(multi_df,"GPT35Finetuned" )

PTSD
Precision  0.0
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.18518518518518517
AUC 0.0
SCHIZOPHRENIA
Precision  1.0
Recall: 0.1111111111111111
F1 0.19999999999999998
TPR 0.1111111111111111
FPR 0.0
AUC 1.0
EATING DISORDER
Precision  1.0
Recall: 0.1111111111111111
F1 0.19999999999999998
TPR 0.1111111111111111
FPR 0.0
AUC 1.0
BIPOLAR
Precision  nan
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.0
AUC nan
AUTISM
Precision  0.5
Recall: 0.2222222222222222
F1 0.30769230769230765
TPR 0.2222222222222222
FPR 0.037037037037037035
AUC 0.8571428571428571
ADHD
Precision  0.14893617021276595
Recall: 0.7777777777777778
F1 0.25
TPR 0.7777777777777778
FPR 0.7407407407407407
AUC 0.5121951219512195
OCD
Precision  nan
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.0
AUC nan


/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:96: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * precision * recall) / (precision + recall )
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:94: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (fp + tp)
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:99: RuntimeWarning: invalid value encountered in scalar divide
  auc = tpr /(tpr+fpr)
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:103: RuntimeWarning: invalid value encountered in scalar divide
  print("Precision " +" " +str( tp/(fp+tp)))
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:108: RuntimeWarning: invalid value encountered in scalar divide
  print("AUC " +str(tpr/ (tpr+fpr)))
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:94: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (fp + tp)
/Users/lukasschmidt/Desktop/Twitter/Performance

,Disorder,Precision,Recall,F1,AUC
0,PTSD,0.000000,0.000000,NaN,0.000000
1,SCHIZOPHRENIA,1.000000,0.111111,0.200000,1.000000
2,EATING DISORDER,1.000000,0.111111,0.200000,1.000000
3,BIPOLAR,NaN,0.000000,NaN,NaN
4,AUTISM,0.500000,0.222222,0.307692,0.857143
5,ADHD,0.148936,0.777778,0.250000,0.512195
6,OCD,NaN,0.000000,NaN,NaN


In [9]:
calculate_performance_multi(multi_df,"Prediction_GPT4_Temp0" )

PTSD
Precision  0.16666666666666666
Recall: 0.1111111111111111
F1 0.13333333333333333
TPR 0.1111111111111111
FPR 0.09259259259259259
AUC 0.5454545454545454
SCHIZOPHRENIA
Precision  0.75
Recall: 0.3333333333333333
F1 0.46153846153846156
TPR 0.3333333333333333
FPR 0.018518518518518517
AUC 0.9473684210526315
EATING DISORDER
Precision  0.125
Recall: 0.2222222222222222
F1 0.16
TPR 0.2222222222222222
FPR 0.25925925925925924
AUC 0.46153846153846156
BIPOLAR
Precision  1.0
Recall: 0.1111111111111111
F1 0.19999999999999998
TPR 0.1111111111111111
FPR 0.0
AUC 1.0
AUTISM
Precision  1.0
Recall: 0.2222222222222222
F1 0.3636363636363636
TPR 0.2222222222222222
FPR 0.0
AUC 1.0
ADHD
Precision  0.0
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.25925925925925924
AUC 0.0
OCD
Precision  nan
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.0
AUC nan


/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:96: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * precision * recall) / (precision + recall )
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:94: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (fp + tp)
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:99: RuntimeWarning: invalid value encountered in scalar divide
  auc = tpr /(tpr+fpr)
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:103: RuntimeWarning: invalid value encountered in scalar divide
  print("Precision " +" " +str( tp/(fp+tp)))
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:108: RuntimeWarning: invalid value encountered in scalar divide
  print("AUC " +str(tpr/ (tpr+fpr)))


,Disorder,Precision,Recall,F1,AUC
0,PTSD,0.166667,0.111111,0.133333,0.545455
1,SCHIZOPHRENIA,0.750000,0.333333,0.461538,0.947368
2,EATING DISORDER,0.125000,0.222222,0.160000,0.461538
3,BIPOLAR,1.000000,0.111111,0.200000,1.000000
4,AUTISM,1.000000,0.222222,0.363636,1.000000
5,ADHD,0.000000,0.000000,NaN,0.000000
6,OCD,NaN,0.000000,NaN,NaN


In [13]:
calculate_performance_multi(multi_df, "GPT35Finetuned_temp")

PTSD
Precision  0.0
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.18518518518518517
AUC 0.0
SCHIZOPHRENIA
Precision  1.0
Recall: 0.1111111111111111
F1 0.19999999999999998
TPR 0.1111111111111111
FPR 0.0
AUC 1.0
EATING DISORDER
Precision  1.0
Recall: 0.1111111111111111
F1 0.19999999999999998
TPR 0.1111111111111111
FPR 0.0
AUC 1.0
BIPOLAR
Precision  nan
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.0
AUC nan
AUTISM
Precision  0.6666666666666666
Recall: 0.2222222222222222
F1 0.3333333333333333
TPR 0.2222222222222222
FPR 0.018518518518518517
AUC 0.9230769230769231
ADHD
Precision  0.14583333333333334
Recall: 0.7777777777777778
F1 0.24561403508771928
TPR 0.7777777777777778
FPR 0.7592592592592593
AUC 0.5060240963855421
OCD
Precision  nan
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.0
AUC nan


/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:96: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * precision * recall) / (precision + recall )
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:94: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (fp + tp)
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:99: RuntimeWarning: invalid value encountered in scalar divide
  auc = tpr /(tpr+fpr)
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:103: RuntimeWarning: invalid value encountered in scalar divide
  print("Precision " +" " +str( tp/(fp+tp)))
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:108: RuntimeWarning: invalid value encountered in scalar divide
  print("AUC " +str(tpr/ (tpr+fpr)))
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:94: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (fp + tp)
/Users/lukasschmidt/Desktop/Twitter/Performance

,Disorder,Precision,Recall,F1,AUC
0,PTSD,0.000000,0.000000,NaN,0.000000
1,SCHIZOPHRENIA,1.000000,0.111111,0.200000,1.000000
2,EATING DISORDER,1.000000,0.111111,0.200000,1.000000
3,BIPOLAR,NaN,0.000000,NaN,NaN
4,AUTISM,0.666667,0.222222,0.333333,0.923077
5,ADHD,0.145833,0.777778,0.245614,0.506024
6,OCD,NaN,0.000000,NaN,NaN


In [10]:
calculate_performance_multi(multi_df,"GPT35TurboBaseTemp0_Multi" )

PTSD
Precision  0.10714285714285714
Recall: 0.3333333333333333
F1 0.16216216216216214
TPR 0.3333333333333333
FPR 0.46296296296296297
AUC 0.41860465116279066
SCHIZOPHRENIA
Precision  1.0
Recall: 0.3333333333333333
F1 0.5
TPR 0.3333333333333333
FPR 0.0
AUC 1.0
EATING DISORDER
Precision  0.125
Recall: 0.1111111111111111
F1 0.11764705882352941
TPR 0.1111111111111111
FPR 0.12962962962962962
AUC 0.46153846153846156
BIPOLAR
Precision  0.0
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.12962962962962962
AUC 0.0
AUTISM
Precision  0.3333333333333333
Recall: 0.3333333333333333
F1 0.3333333333333333
TPR 0.3333333333333333
FPR 0.1111111111111111
AUC 0.75
ADHD
Precision  0.0
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.018518518518518517
AUC 0.0
OCD
Precision  0.0
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.05555555555555555
AUC 0.0


/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:96: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * precision * recall) / (precision + recall )
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:96: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * precision * recall) / (precision + recall )
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:96: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * precision * recall) / (precision + recall )


,Disorder,Precision,Recall,F1,AUC
0,PTSD,0.107143,0.333333,0.162162,0.418605
1,SCHIZOPHRENIA,1.000000,0.333333,0.500000,1.000000
2,EATING DISORDER,0.125000,0.111111,0.117647,0.461538
3,BIPOLAR,0.000000,0.000000,NaN,0.000000
4,AUTISM,0.333333,0.333333,0.333333,0.750000
5,ADHD,0.000000,0.000000,NaN,0.000000
6,OCD,0.000000,0.000000,NaN,0.000000


In [11]:
calculate_performance_multi(multi_df,"GPT35TurboBaseTopPMulti" )

PTSD
Precision  0.125
Recall: 0.3333333333333333
F1 0.18181818181818182
TPR 0.3333333333333333
FPR 0.3888888888888889
AUC 0.4615384615384615
SCHIZOPHRENIA
Precision  0.5
Recall: 0.2222222222222222
F1 0.30769230769230765
TPR 0.2222222222222222
FPR 0.037037037037037035
AUC 0.8571428571428571
EATING DISORDER
Precision  0.1111111111111111
Recall: 0.1111111111111111
F1 0.1111111111111111
TPR 0.1111111111111111
FPR 0.14814814814814814
AUC 0.42857142857142855
BIPOLAR
Precision  0.0
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.14814814814814814
AUC 0.0
AUTISM
Precision  0.42857142857142855
Recall: 0.3333333333333333
F1 0.375
TPR 0.3333333333333333
FPR 0.07407407407407407
AUC 0.8181818181818182
ADHD
Precision  0.0
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.018518518518518517
AUC 0.0
OCD
Precision  0.0
Recall: 0.0
F1 nan
TPR 0.0
FPR 0.12962962962962962
AUC 0.0


/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:96: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * precision * recall) / (precision + recall )
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:96: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * precision * recall) / (precision + recall )
/Users/lukasschmidt/Desktop/Twitter/PerformanceAnalyzer.py:96: RuntimeWarning: invalid value encountered in scalar divide
  f1 = (2 * precision * recall) / (precision + recall )


,Disorder,Precision,Recall,F1,AUC
0,PTSD,0.125000,0.333333,0.181818,0.461538
1,SCHIZOPHRENIA,0.500000,0.222222,0.307692,0.857143
2,EATING DISORDER,0.111111,0.111111,0.111111,0.428571
3,BIPOLAR,0.000000,0.000000,NaN,0.000000
4,AUTISM,0.428571,0.333333,0.375000,0.818182
5,ADHD,0.000000,0.000000,NaN,0.000000
6,OCD,0.000000,0.000000,NaN,0.000000


In [14]:
multi_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,_id,sampled_tweets,ground_truth,binary_disorder,GPT35Finetuned,Prediction_GPT4_Temp0,GPT35TurboBaseTemp0_Multi,GPT35TurboBaseTopPMulti,GPT35Finetuned_temp
0,0,0,1,2.716031e+06,"""great i have... another? the same? ear infect...",ADHD,psychological disorder,ADHD,"I'm sorry, but I cannot provide a diagnosis or...",PTSD,BIPOLAR,ADHD
1,1,1,2,8.481222e+06,"""People should always come before profits. Bec...",ADHD,psychological disorder,PTSD,"I'm sorry, but it is not appropriate or ethica...",PTSD,PTSD,PTSD
2,2,2,3,1.010535e+07,"""Black lives still matter. COVID isnt over. ...",ADHD,psychological disorder,ADHD,"I'm sorry, but I cannot provide a diagnosis or...",PTSD,PTSD,ADHD
3,3,3,4,1.422955e+07,"""@USER I love the ride but that pre show and t...",ADHD,psychological disorder,ADHD,EATING DISORDER,BIPOLAR,AUTISM,ADHD
4,4,4,5,1.439176e+07,"""@USER I am 100% certain this is exactly what ...",ADHD,psychological disorder,ADHD,"I'm sorry, but I cannot provide a diagnosis or...",PTSD,PTSD,ADHD
...,...,...,...,...,...,...,...,...,...,...,...,...
58,58,58,94,2.314623e+09,"""@USER Its about the schiz. You know the 7/22 ...",SCHIZOPHRENIA,psychological disorder,SCHIZOPHRENIA,SCHIZOPHRENIA,SCHIZOPHRENIA,SCHIZOPHRENIA,SCHIZOPHRENIA
59,59,59,96,1.404918e+18,"""I like how my brother's friends always have h...",SCHIZOPHRENIA,psychological disorder,PTSD,EATING DISORDER,PTSD,PTSD,PTSD
60,60,60,97,2.377368e+09,"""""Elevated odds of multiple conditions at the ...",SCHIZOPHRENIA,psychological disorder,ADHD,SCHIZOPHRENIA,PTSD,PTSD,ADHD
61,61,61,98,8.010677e+08,"""jesse x bullseye otp"";""now, i'm no homophobe""...",SCHIZOPHRENIA,psychological disorder,ADHD,"I'm sorry, but I cannot assist with this request.",SCHIZOPHRENIA,OCD,ADHD


In [16]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,_id,sampled_tweets,ground_truth,binary_disorder,GPT35Finetuned_Binary,GPT35TurboBaseTopP1_Binary,GPT35TurboBaseTemp0_Binary
0,0,0,1,2.716031e+06,"""great i have... another? the same? ear infect...",ADHD,psychological disorder,psychological disorder,psychological disorder,psychological disorder
1,1,1,2,8.481222e+06,"""People should always come before profits. Bec...",ADHD,psychological disorder,no psychological disorder,no psychological disorder,psychological disorder
2,2,2,3,1.010535e+07,"""Black lives still matter. COVID isnt over. ...",ADHD,psychological disorder,no psychological disorder,no psychological disorder,psychological disorder
3,3,3,4,1.422955e+07,"""@USER I love the ride but that pre show and t...",ADHD,psychological disorder,psychological disorder,psychological disorder,psychological disorder
4,4,4,5,1.439176e+07,"""@USER I am 100% certain this is exactly what ...",ADHD,psychological disorder,no psychological disorder,no psychological disorder,psychological disorder
...,...,...,...,...,...,...,...,...,...,...
85,85,85,94,2.314623e+09,"""@USER Its about the schiz. You know the 7/22 ...",SCHIZOPHRENIA,psychological disorder,psychological disorder,psychological disorder,psychological disorder
86,86,86,96,1.404918e+18,"""I like how my brother's friends always have h...",SCHIZOPHRENIA,psychological disorder,psychological disorder,psychological disorder,psychological disorder
87,87,87,97,2.377368e+09,"""""Elevated odds of multiple conditions at the ...",SCHIZOPHRENIA,psychological disorder,no psychological disorder,no psychological disorder,psychological disorder
88,88,88,98,8.010677e+08,"""jesse x bullseye otp"";""now, i'm no homophobe""...",SCHIZOPHRENIA,psychological disorder,psychological disorder,psychological disorder,psychological disorder
